In [11]:
##############################################################################################
##                                                                                          ##
##          simple SDF Browser                                                              ##
##                                                                                          ##
##############################################################################################


import ipywidgets as widgets
from IPython.display import display, clear_output

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit import RDLogger
import py3Dmol

# supress RDKit-messages
RDLogger.DisableLog('rdApp.*')  # Deactivate all RDKit warnings and messages



##########################################################
#                                                        #
#              Enter your filename here                  #
#                                                        #
##########################################################


# read molecules from sdf
input_sdf = 'output-test.sdf'  # Ersetze dies durch den Pfad zu deiner SDF-Datei



####   do not edit #####


supplier = Chem.SDMolSupplier(input_sdf)
molecules = [mol for mol in supplier if mol is not None]

# are molecules loaded?
if not molecules:
    print("sdf does not contain molecules.")
else:
    print(f"{len(molecules)} molecules loaded.")
    

# display molecul in 3D
def display_molecule_3d(index):
    if index < 0 or index >= len(molecules):
        print(f"Ungültiger Index. Bitte wähle eine Nummer zwischen 0 und {len(molecules) - 1}.")
        return None
    
    mol = molecules[index]
    try:
        # 3D-Koordinaten berechnen, falls nicht vorhanden
        if not mol.GetNumConformers():
            AllChem.Compute2DCoords(mol)  # Fallback auf 2D, falls 3D nicht funktioniert
            AllChem.EmbedMolecule(mol, randomSeed=42)  # 3D-Koordinaten generieren
        
        # Molekül in PDB-Format konvertieren für py3Dmol
        pdb = Chem.MolToPDBBlock(mol)
        
        # 3D-Viewer erstellen
        viewer = py3Dmol.view(width=400, height=300)
        viewer.addModel(pdb, 'pdb')
        viewer.setStyle({'stick': {}})
        viewer.zoomTo()
        return viewer
    except Exception as e:
        print(f"error rendering molecule: {e}")
        return None
    
    

# initialise index
eingabe = 0

# show index value
def show_value():
    clear_output(wait=True)
    print(f"actual molecule shown: {eingabe}")
    viewer=display_molecule_3d(eingabe)
    
    viewer.show()
    display(widgets.HBox([button_decrease, button_increase]))  # show buttons

# increase index
def increase_value(b):
    global eingabe
    eingabe += 1
    show_value()

# decrease index
def decrease_value(b):
    global eingabe
    eingabe -= 1
    show_value()

# create buttons
button_increase = widgets.Button(description="next molecule")
button_decrease = widgets.Button(description="previous molecule")

# actions for buttons
button_increase.on_click(increase_value)
button_decrease.on_click(decrease_value)

# initial display
show_value()


actual molecule shown: 3


3Dmol.js failed to load for some reason. Please check your browser console for error messages.